In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from docplex.mp.model import  Model

source: 
https://www.linkedin.com/pulse/optimizing-marketing-portfolio-through-linear-neelima-gaddam/

Assume that ROI from each campaign as below: 7% from TV, 3% from Print, 15% from Mobile Advertising, 12% from SEO + Adwords, and 5% from Facebook.


1. Production and airing the TV advertisements cost at least $500K.

2. Print media should not account for more than 5% of the budget.
    
3. Minimum Ad agency costs for content creation and placement for Billboards, game place promotions, airport promotion cost $200K.
    
4. Mobile and SEO + AdWords together should account for at least 50% and that SEO+Adwords should be no more than 2.5 times the mobile marketing costs.
    
5. Mobile content and advertising cost anywhere between $300K and $900K.

6. Facebook advertising should not exceed 15 % of the marketing spend.

7. Agency costs for Facebook marketing cost at least $100K.


It has been learnt that a $1 spent on each channel has customer reach as: Channel (# customers reached), TV (2), Print (0.3), Mobile (1.8), SEO+Adwords (0.9), Facebook (2).


marketing budget should not exceed $5 million and there are about 2.5 million customers in the target market segment.

In [8]:
channels = pd.DataFrame( [[7 , 'TV'], [3 , 'Print'], [15 , 'Mobile Advertising'], [12 , 'SEO Adwords'], [ 5 , 'Facebook']], columns = ['ROI','channel'] )
channels['Reach']=[2,0.3,1.8,0.9, 2]

channels

,ROI,channel,Reach
0,7,TV,2.0
1,3,Print,0.3
2,15,Mobile Advertising,1.8
3,12,SEO Adwords,0.9
4,5,Facebook,2.0


In [32]:
mdl = Model ("Marketing")

In [33]:
channels['BudgetDV'] = 0

for index, row in channels.iterrows():
    dv =  mdl.continuous_var(lb = 0 , name="X_Budget:{}".format(row.channel))
    channels.at[index, 'BudgetDV']=dv


channels


,ROI,channel,Reach,BudgetDV
0,7,TV,2.0,X_Budget:TV
1,3,Print,0.3,X_Budget:Print
2,15,Mobile Advertising,1.8,X_Budget:Mobile Advertising
3,12,SEO Adwords,0.9,X_Budget:SEO Adwords
4,5,Facebook,2.0,X_Budget:Facebook


Total budget should not exceed $5 Million

In [34]:
Total_Budgets = mdl.sum(channels['BudgetDV'])

mdl.add_kpi(Total_Budgets   , "Total_Budgets")

DecisionKPI(name=Total_Budgets,expr=X_Budget:TV+X_Budget:Print+X_Budget:Mobile Advertising+X_Budget:..)

In [35]:
mdl.add ( Total_Budgets <= 5000) ;

TV advertising costs at least $500K

In [36]:
mdl.add ( mdl.sum(channels[channels['channel']=='TV']['BudgetDV']) >= 500 );

Print media not greater than 5% of the budget

In [37]:
mdl.add ( mdl.sum(channels[channels['channel']=='Print']['BudgetDV'])<= 0.05 * Total_Budgets );

Print media costs at least $200K 

In [38]:
mdl.add ( mdl.sum(channels[channels['channel']=='Print']['BudgetDV']) >= 200)

docplex.mp.LinearConstraint[](X_Budget:Print,GE,200)

Mobile and SEO + AdWords to account for at least 50% of budget 

In [39]:
mdl.add (  mdl.sum(  channels[channels['channel'].isin(['Mobile Advertising','SEO Adwords'])]['BudgetDV'] ) >= 0.5 * Total_Budgets  );

SEO + AdWords to be not more than 2.5 times of Mobile

In [40]:
mdl.add (   mdl.sum( channels[channels['channel'].isin(['Mobile Advertising'])]['BudgetDV'] ) <= 2.5 * mdl.sum(channels[channels['channel']=='SEO Adwords']['BudgetDV'])  )

docplex.mp.LinearConstraint[](X_Budget:Mobile Advertising,LE,2.500X_Budget:SEO Adwords)

In [41]:
#x3 = mdl.sum( channels[channels['channel'].isin(['Mobile Advertising','SEO Adwords'])]['BudgetDV'])
#x4 = mdl.sum(channels[channels['channel']=='Mobile Advertising']['BudgetDV']) 
#mdl.add (  x4 <= 2.5 * x3 )

Mobile marketing budget between $300K and $900K 

In [42]:
mdl.add (300 <= mdl.sum(channels[channels['channel']=='SEO Adwords']['BudgetDV']) )

mdl.add (mdl.sum(channels[channels['channel']=='SEO Adwords']['BudgetDV']) <= 900 )

docplex.mp.LinearConstraint[](X_Budget:SEO Adwords,LE,900)

Facebook advertising costs at least $100K —> x5 >= 100K 

In [43]:
mdl.add (mdl.sum(channels[channels['channel']=='Facebook']['BudgetDV']) >= 100 )

docplex.mp.LinearConstraint[](X_Budget:Facebook,GE,100)

Reaching 2.5 million customers

In [44]:
mdl.add (  mdl.sum( channels['Reach']*channels['BudgetDV'] )  <= 2500  )

docplex.mp.LinearConstraint[](2X_Budget:TV+0.300X_Budget:Print+1.800X_Budget:Mobile Advertising+0.900X_Budget:SEO Adwords+2X_Budget:Facebook,LE,2500)

In [45]:
Total_Profit = mdl.sum(channels['BudgetDV'] * channels['ROI']*0.01)

mdl.add_kpi(Total_Profit   , "Total_Profit")

DecisionKPI(name=Total_Profit,expr=0.070X_Budget:TV+0.030X_Budget:Print+0.150X_Budget:Mobile Advert..)

In [46]:
mdl.maximize(Total_Profit)
msol = mdl.solve(log_output=True)

mdl.report()

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Implied bounds make row 'c3' infeasible.
Presolve time = 0.00 sec. (0.00 ticks)


In [47]:
channels['BudgetDV_sol']= channels['BudgetDV'].apply(lambda x:x.solution_value)

channels

DOcplexException: Model<Marketing> did not solve successfully

In [48]:
from docplex.mp.conflict_refiner import ConflictRefiner
cr = ConflictRefiner()
crr = cr.refine_conflict(mdl, display=True)

from docplex.mp.relaxer import Relaxer
rx = Relaxer()
rs = rx.relax(mdl)
print(rs)

conflicts: 6
  - linear constraints: 6
solution for: Marketing
objective: 189.833
status: INFEASIBLE_SOLUTION(3)
X_Budget:TV=500.000
X_Budget:Print=200.000
X_Budget:Mobile Advertising=238.889
X_Budget:SEO Adwords=900.000
X_Budget:Facebook=100.000



In [107]:
print( mdl.export_as_lp_string() )

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Marketing

Maximize
 obj: 0.070000000000 x1 + 0.030000000000 x2 + 0.150000000000 x3
      + 0.120000000000 x4 + 0.050000000000 x5
Subject To
 c1: 2 x1 + 0.300000000000 x2 + 1.800000000000 x3 + 0.900000000000 x4 + 2 x5
      <= 2500
 c2: x1 + x2 + x3 + x4 + x5 <= 5000
 c3: x1 >= 500
 c4: 0.950000000000 x2 - 0.050000000000 x1 - 0.050000000000 x3
     - 0.050000000000 x4 - 0.050000000000 x5 <= 0
 c5: x2 >= 200

Bounds
End



In [112]:
channels['BudgetDV'] = 0


dv

[docplex.mp.Var(type=C,name='X_1'),
 docplex.mp.Var(type=C,name='X_2'),
 docplex.mp.Var(type=C,name='X_3'),
 docplex.mp.Var(type=C,name='X_4'),
 docplex.mp.Var(type=C,name='X_5')]

In [7]:
mdl = Model ("Marketing")
dv =  mdl.continuous_var_list(keys = range(1,6) , lb = 0 , name="X")

mdl.add ( mdl.scal_prod(dv, [1,1,1,1,1] )<= 5000 ) 
mdl.add ( mdl.scal_prod(dv, [-1, 0, 0, 0, 0] )<= -500 ) 
mdl.add ( mdl.scal_prod(dv, [-0.05, 0.95, -0.05, -0.05, -0.05] )<= 0 ) 
mdl.add ( mdl.scal_prod(dv, [-0, -1, 0, 0, 0] )<= -200 ) 
mdl.add ( mdl.scal_prod(dv, [0.5, 0.5, -0.5, -0.5, 0.5] )<= 0 ) 
mdl.add ( mdl.scal_prod(dv, [0, 0, 1, -2.5, 0] )<= 0 ) 
mdl.add ( mdl.scal_prod(dv, [0, 0, 0, -1, 0] )<=-300 ) 
mdl.add ( mdl.scal_prod(dv, [0, 0, 0, 1, 0] )<=900 ) 
mdl.add ( mdl.scal_prod(dv, [0, 0, 0, 0, -1] )<=-100 ) 
mdl.add ( mdl.scal_prod(dv, [2, 0.3, 1.8, 0.9, 2] )<=2500 ) 


mdl.maximize(mdl.scal_prod(dv, [0.07, 0.03, 0.15, 0.12, 0.05] ))
msol = mdl.solve(log_output=True)

mdl.report()

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Implied bounds make row 'c3' infeasible.
Presolve time = 0.00 sec. (0.00 ticks)


In [27]:
from docplex.mp.conflict_refiner import ConflictRefiner
cr = ConflictRefiner()
crr = cr.refine_conflict(mdl, display=True)

from docplex.mp.relaxer import Relaxer
rx = Relaxer()
rs = rx.relax(mdl)

conflicts: 6
  - linear constraints: 6


In [31]:
print(rs)

solution for: Marketing
objective: 189.833
status: INFEASIBLE_SOLUTION(3)
X_Budget:TV=500.000
X_Budget:Print=200.000
X_Budget:Mobile Advertising=238.889
X_Budget:SEO Adwords=900.000
X_Budget:Facebook=100.000



In [3]:
def scal_prod(x,y):
    return sum([a*b for a,b in zip(x,y)])

##[i.solution_value for i in dv]
dv = [500, 92.4638, 256.8116, 900.0000,100.0000 ]
print(scal_prod(dv, [1,1,1,1,1] ), 5000 )
print(scal_prod(dv, [-1, 0, 0, 0, 0] ), -500 ) 
print(scal_prod(dv, [-0.05, 0.95, -0.05, -0.05, -0.05] ), 0 ) 
print(scal_prod(dv, [-0.01, -1, 0, 0, 0] ), -200 ) 
print(scal_prod(dv, [0.5, 0.5, -0.5, -0.5, 0.5] ), 0 ) 
print(scal_prod(dv, [0, 0, 1, -2.5, 0] ), 0 ) 
print(scal_prod(dv, [0, 0, 0, -1, 0] ),-300 ) 
print(scal_prod(dv, [0, 0, 0, 1, 0] ),900 ) 
print(scal_prod(dv, [0, 0, 0, 0, -1] ),-100 ) 
print(scal_prod(dv, [2, 0.3, 1.8, 0.9, 2] ),2500 ) 

1849.2754 5000
-500.0 -500
2.9999999995311555e-05 0
-97.4638 -200
-232.1739 0
-1993.1884 0
-900.0 -300
900.0 900
-100.0 -100
2500.00002 2500


In [1]:
library(linprog)
 
Max_ROI <- c(0.07, 0.03, 0.15, 0.12, 0.05) #Objective Function
 
Contraint_Vector <- c(5000, -500, 0, -200, 0, 0, -300, 900, -100, 2500) #Constraints

#Decision variables under constraints 
Decision_Var <- rbind(
  c(1,1,1,1,1), 
  c(-1, 0, 0, 0, 0), 
  c(-0.05, 0.95, -0.05, -0.05, -0.05), 
  c(-0, -1, 0, 0, 0), 
  c(0.5, 0.5, -0.5, -0.5, 0.5), 
  c(0, 0, 1, -2.5, 0), 
  c(0, 0, 0, -1, 0), 
  c(0, 0, 0, 1, 0), 
  c(0, 0, 0, 0, -1),
  c(2, 0.3, 1.8, 0.9, 2)
)
 
solveLP(Max_ROI, Contraint_Vector, Decision_Var, TRUE)

Loading required package: lpSolve





Results of Linear Programming / Linear Optimization

Objective function (Maximum): 189.296 

Iterations in phase 1: 7
Iterations in phase 2: 0
Solution
       opt
1 500.0000
2  92.4638
3 256.8116
4 900.0000
5 100.0000

Basic Variables
          opt
1    500.0000
2     92.4638
3    256.8116
4    900.0000
5    100.0000
S 1 3150.7246
S 4  107.5362
S 5  232.1739
S 6 1993.1884
S 7  600.0000

Constraints
      actual dir bvec     free       dual  dual.reg
1   1849.275  <= 5000 3150.725 0.00000000 3150.7246
2   -500.000  <= -500    0.000 0.09669565  220.6612
3      0.000  <=    0    0.000 0.00521739   88.6111
4   -307.536  <= -200  107.536 0.00000000  107.5362
5   -232.174  <=    0  232.174 0.00000000  232.1739
6  -1993.188  <=    0 1993.188 0.00000000 1993.1884
7   -900.000  <= -300  600.000 0.00000000  600.0000
8    900.000  <=  900    0.000 0.04513043  600.0000
9   -100.000  <= -100    0.000 0.11669565  220.6612
10  2500.000  <= 2500    0.000 0.08347826  466.3158

All Variables (includin